In [1]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

from datasets import load_dataset
imdb = load_dataset("imdb")

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
 
tokenized_train = imdb["train"].shuffle(seed=42).map(preprocess_function, batched=True)
tokenized_test = imdb["test"].shuffle(seed=42).map(preprocess_function, batched=True)

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [18]:
costs = [len(imdb["train"][i]['text']) for i in range(len(imdb["train"]))]
# embeddings = [model.encode(imdb["train"][i]['text'], convert_to_tensor=True) for i in range(len(imdb["train"]))]

In [6]:
texts = [imdb["train"][i]['text'] for i in range(len(imdb["train"]))]

In [11]:
embeddings = model.encode(texts, convert_to_tensor=True)

In [12]:
embeddings.shape

torch.Size([25000, 384])

In [13]:
similarity = model.similarity(embeddings,embeddings)

In [15]:
similarity.shape

torch.Size([25000, 25000])